In [1]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from math import log, ceil
import time
import sys
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [2]:
import pickle
with open("Dane/wiki_list", "rb") as ifile:
    wiki_list = pickle.load(ifile)

In [3]:
def to_lemma_mapping():
    all_lemmas = dd(list)
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[:2]
        all_lemmas[L[1].lower()].append(L[0].lower())
    return all_lemmas

In [4]:
with open("Dane/polish.stopwords.txt", "r") as ifile:
    stopwords = [line[:-1] for line in ifile]

In [5]:
stopwords

['a',
 'aby',
 'ach',
 'acz',
 'aczkolwiek',
 'aj',
 'albo',
 'ale',
 'alez',
 'ależ',
 'ani',
 'az',
 'aż',
 'bardziej',
 'bardzo',
 'beda',
 'bedzie',
 'bez',
 'deda',
 'będą',
 'bede',
 'będę',
 'będzie',
 'bo',
 'bowiem',
 'by',
 'byc',
 'być',
 'byl',
 'byla',
 'byli',
 'bylo',
 'byly',
 'był',
 'była',
 'było',
 'były',
 'bynajmniej',
 'cala',
 'cali',
 'caly',
 'cała',
 'cały',
 'ci',
 'cie',
 'ciebie',
 'cię',
 'co',
 'cokolwiek',
 'cos',
 'coś',
 'czasami',
 'czasem',
 'czemu',
 'czy',
 'czyli',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokad',
 'dokąd',
 'dosc',
 'dość',
 'duzo',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzis',
 'dzisiaj',
 'dziś',
 'gdy',
 'gdyby',
 'gdyz',
 'gdyż',
 'gdzie',
 'gdziekolwiek',
 'gdzies',
 'gdzieś',
 'go',
 'i',
 'ich',
 'ile',
 'im',
 'inna',
 'inne',
 'inny',
 'innych',
 'iz',
 'iż',
 'ja',
 'jak',
 'jakas',
 'jakaś',
 'jakby',
 'jaki',
 'jakichs',
 'jakichś',
 'jakie',
 'jakis',
 'jakiś',
 'jakiz',
 'jakiż',
 'jak

In [6]:
lemmas = to_lemma_mapping()

In [7]:
lemmas["zielonej"]

['zielona', 'zielony']

In [8]:
wiki_list[0]

('AWK',
 [['AWK'],
  ['AWK',
   '–',
   'interpretowany',
   'język',
   'programowania',
   ',',
   'którego',
   'główną',
   'funkcją',
   'jest',
   'wyszukiwanie',
   'i',
   'przetwarzanie',
   'wzorców',
   'w',
   'plikach',
   'lub',
   'strumieniach',
   'danych',
   '.',
   'Jest',
   'także',
   'nazwą',
   'programu',
   'początkowo',
   'dostępnego',
   'dla',
   'systemów',
   'operacyjnych',
   'będących',
   'pochodnymi',
   'UNIX-a',
   ',',
   'obecnie',
   'także',
   'na',
   'inne',
   'platformy',
   '.'],
  ['AWK',
   'jest',
   'językiem',
   ',',
   'który',
   'w',
   'znacznym',
   'stopniu',
   'wykorzystuje',
   'tablice',
   'asocjacyjne',
   ',',
   'stringi',
   'i',
   'wyrażenia',
   'regularne',
   '.',
   'Nazwa',
   'języka',
   'pochodzi',
   'od',
   'pierwszych',
   'liter',
   'nazwisk',
   'jego',
   'autorów',
   'Alfreda',
   'V.',
   'Aho',
   ',',
   'Petera',
   'Weinbergera',
   'i',
   'Briana',
   'Kernighana',
   'i',
   'czasami',
  

In [9]:
test = dd(lambda:0)

In [10]:
"a" in test

False

In [ ]:
corpus = []
for title, article in wiki_list:
    art_processed = []
    for line in article:
        for w in line:
            w = w.lower()
            if w in lemmas:
                art_processed.extend(lemmas[w])
            else:
                art_processed.append(w)
    corpus.append(" ".join(art_processed))

In [ ]:
corpus = []
key = ""
with open("Dane/fp_wiki.txt") as f:
    cntr = -1
    for line in f:
        tokenized_line = line.split(": ")
        if tokenized_line[0] == "TITLE":
            cntr+=1
            corpus.append([])
            continue
        corpus[-1].append(line)

In [13]:
corpus = ["".join(el) for el in corpus]

In [14]:
corpus[:3]

["AWK\nAWK – interpretowany język programowania , którego główną funkcją jest wyszukiwanie i przetwarzanie wzorców w plikach lub strumieniach danych . Jest także nazwą programu początkowo dostępnego dla systemów operacyjnych będących pochodnymi UNIX-a , obecnie także na inne platformy .\nAWK jest językiem , który w znacznym stopniu wykorzystuje tablice asocjacyjne , stringi i wyrażenia regularne . Nazwa języka pochodzi od pierwszych liter nazwisk jego autorów Alfreda V. Aho , Petera Weinbergera i Briana Kernighana i czasami jest zapisywana małymi literami oraz odczytywana jako jedno słowo `` awk '' .\n\n",
 'Alergologia\nAlergologia – dziedzina medycyny zajmująca się rozpoznawaniem i leczeniem schorzeń alergicznych .\n\n',
 "ASCII\nASCII ( czyt . `` aski ; `` ang . `` American Standard Code for Information Interchange '' ) – 7-bitowy kod przyporządkowujący liczby z zakresu 0−127 : literom alfabetu angielskiego , cyfrom , znakom przestankowym i innym symbolom oraz poleceniom sterującym 

In [15]:
def process_word(w):
    w = w.lower()
    if w in lemmas:
        return " ".join(lemmas[w])
    return w

In [16]:
vectorizer = TfidfVectorizer(stop_words = stopwords)

In [17]:
X = vectorizer.fit_transform(corpus)

In [18]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree', metric=cosine_distances, n_jobs=50).fit(X)


/home/mikolaj/miniconda3/envs/IoT-Pathway/lib/python3.7/site-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [19]:
distances, indices = nbrs.kneighbors(X[:2])

In [ ]:
import pickle
with open("Dane/indices.pickle", "wb") as ofile:
    pickle.dump(indices, ofile)

In [1]:
"test"

'test'

In [2]:
indices

NameError: name 'indices' is not defined